In [2]:
import csv
import os
import requests as req
import json
from collections import Counter

In [3]:
#Donorschoose.org API
api_key = "DONORSCHOOSE"
url = "https://api.donorschoose.org/common/json_feed.html?max=100"

In [4]:
#Open a CSV for a list of Bay Area cities
with open("bayarea_cities.csv") as cities:
    cities_reader = csv.reader(cities)
    
    cities = [city[0] for city in cities_reader]
    
    print(len(cities))

cities = cities[13:22]
print(cities)

102
['Calistoga', 'Campbell', 'Clayton', 'Cloverdale', 'Colma', 'Concord', 'Corte Madera', 'Cotati', 'Cupertino']


In [5]:
# Build partial query URL
query_url = url + "&APIkey=" + api_key + "&state=CA" + "&cityName="
#location_data = [req.get(query_url + city).json() for city in cities]

all_data = {}

for city in cities:
    
    results = req.get(query_url + city).json()
    all_data[city] = results   

print(json.dumps(all_data, indent=2))

{
  "Calistoga": {
    "searchTerms": "CA",
    "searchURL": "https://www.donorschoose.org/donors/search.html?APIkey=DONORSCHOOSE&cityName=Calistoga&state=CA&utm_source=api&utm_medium=feed&utm_content=searchlink&utm_campaign=DONORSCHOOSE",
    "totalProposals": "0",
    "index": "0",
    "max": "50",
    "breadcrumb": [
      [
        "max",
        "100",
        ""
      ],
      [
        "state",
        "CA",
        "California"
      ]
    ],
    "proposals": []
  },
  "Campbell": {
    "searchTerms": "CA",
    "searchURL": "https://www.donorschoose.org/donors/search.html?APIkey=DONORSCHOOSE&cityName=Campbell&state=CA&utm_source=api&utm_medium=feed&utm_content=searchlink&utm_campaign=DONORSCHOOSE",
    "totalProposals": "0",
    "index": "0",
    "max": "50",
    "breadcrumb": [
      [
        "max",
        "100",
        ""
      ],
      [
        "state",
        "CA",
        "California"
      ]
    ],
    "proposals": []
  },
  "Clayton": {
    "searchTerms": "CA",
    

In [6]:
city_data = {}

for cityname, data in all_data.items():
        
    #step 1: get cost to complete
    #step 2: get povertyType label
    #step 3: add this to the output vairable
    
    costcomplete = []
    povertytype = []
    
    data["proposals"]
    
    for x in data["proposals"]:
        
        costcomplete.append(float(x["costToComplete"]))
        povertytype.append(x["povertyType"]["label"])
    
    city_data[cityname] = {
        
        "costToComplete": sum(costcomplete),
        "povertyType": Counter(povertytype)
    }

city_data

{'Calistoga': {'costToComplete': 0, 'povertyType': Counter()},
 'Campbell': {'costToComplete': 0, 'povertyType': Counter()},
 'Clayton': {'costToComplete': 14776.599999999999,
  'povertyType': Counter({'LOW': 19})},
 'Cloverdale': {'costToComplete': 0, 'povertyType': Counter()},
 'Colma': {'costToComplete': 0, 'povertyType': Counter()},
 'Concord': {'costToComplete': 26611.239999999998,
  'povertyType': Counter({'HIGH': 5,
           'HIGHEST': 13,
           'LOW': 7,
           'MODERATEHIGH': 20,
           'MODERATELOW': 5})},
 'Corte Madera': {'costToComplete': 0, 'povertyType': Counter()},
 'Cotati': {'costToComplete': 1463.1,
  'povertyType': Counter({'MODERATEHIGH': 3})},
 'Cupertino': {'costToComplete': 99.65, 'povertyType': Counter({'LOW': 1})}}

In [7]:
#list of cityname, costtocomplete, number of low, number of modeatelow, etc.

csv_data = []

citynames = sorted(city_data.keys())

for cityname in citynames: 
    
    data = city_data[cityname]
    
    one_row = []
    
    one_row.append(cityname)
    one_row.append(data['costToComplete'])  
    
    labeltype = ["LOW", "MODERATELOW", "MODERATE", "MODERATEHIGH", "HIGH", "HIGHEST", "UNKNOWN"]
    
    for x in labeltype:
        
        one_row.append(data['povertyType'].get(x, 0))
    
    csv_data.append(one_row)

csv_data

[['Calistoga', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Campbell', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Clayton', 14776.599999999999, 19, 0, 0, 0, 0, 0, 0],
 ['Cloverdale', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Colma', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Concord', 26611.239999999998, 7, 5, 0, 20, 5, 13, 0],
 ['Corte Madera', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Cotati', 1463.1, 0, 0, 0, 3, 0, 0, 0],
 ['Cupertino', 99.65, 1, 0, 0, 0, 0, 0, 0]]

In [8]:
root_path = os.path.join(os.getcwd())
data_path = os.path.join(root_path)
csvpath = os.path.join(root_path, "donorschoose_api.csv")
with open(csvpath, "a", newline="") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=",")
    csvwriter.writerows(csv_data)